In [3]:
# Write to a new SQLite file to avoid the lock.
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path


# --- Configuration & Setup ---
sqlite_path = Path("datadim_dates_v2.sqlite")

# --- Brutal Drop of Existing Table (Development Only) ---
# This block connects to the database and unconditionally drops the dim_dates
# table if it already exists. This ensures a clean slate for every script run.
# This is prominent at the top as a reminder to replace it with a more
# robust data management strategy in a production environment.
try:
    print(f"Checking for and removing existing '{sqlite_path}' table...")
    conn_cleanup = sqlite3.connect(sqlite_path)
    cur_cleanup = conn_cleanup.cursor()
    cur_cleanup.execute("DROP TABLE IF EXISTS dim_dates;")
    conn_cleanup.commit()
    print("Table removed successfully.")
except sqlite3.Error as e:
    print(f"An error occurred during cleanup: {e}")
finally:
    if 'conn_cleanup' in locals() and conn_cleanup:
        conn_cleanup.close()
# --- End of Cleanup Block ---

# Feature Engineering
df["date_id"] = pd.Series(rng.strftime("%Y%m%d")).astype(int)
df["day_of_week"] = rng.weekday + 1
df["day_name"] = rng.strftime('%a').str.upper() # Using upper-case abbreviations
df["is_weekend"] = (df["day_of_week"] >= 6).astype(int)
df["year"] = rng.year.astype(int)
df["quarter"] = rng.quarter.astype(int)
df["month"] = rng.month.astype(int)
df["day"] = rng.day.astype(int)
df["month_name"] = rng.month_name()
df["year_month"] = (df["year"] * 100 + df["month"]).astype(int)
df["is_month_end"] = rng.is_month_end.astype(int)
df["is_quarter_end"] = rng.is_quarter_end.astype(int)
df["is_year_end"] = rng.is_year_end.astype(int)
iso = rng.isocalendar()
df["iso_year"] = iso["year"].values.astype(int)
df["iso_week"] = iso["week"].values.astype(int)
df["is_week_start"] = (df["day_of_week"] == 1).astype(int)
df["is_week_end"] = (df["day_of_week"] == 7).astype(int)
df["is_business_day_generic"] = (~(df["day_of_week"] >= 6)).astype(int)
df["business_day_seq_generic"] = df["is_business_day_generic"].cumsum().astype(int)

cols = [
  "date","date_id","day_of_week","day_name","is_weekend","year","quarter","month","day",
  "month_name","year_month","is_month_end","is_quarter_end","is_year_end",
  "iso_year","iso_week","is_week_start","is_week_end","is_business_day_generic","business_day_seq_generic"
]

# --- Database Population ---
print("Connecting to database and populating table...")
conn = sqlite3.connect(sqlite_path)
cur = conn.cursor()

# Create Table
cur.executescript("""
CREATE TABLE IF NOT EXISTS dim_dates (
  date                      DATE PRIMARY KEY,
  date_id                   INTEGER NOT NULL UNIQUE,
  day_of_week               INTEGER NOT NULL,
  day_name                  TEXT NOT NULL,
  is_weekend                INTEGER NOT NULL,
  year                      INTEGER NOT NULL,
  quarter                   INTEGER NOT NULL,
  month                     INTEGER NOT NULL,
  day                       INTEGER NOT NULL,
  month_name                TEXT NOT NULL,
  year_month                INTEGER NOT NULL,
  is_month_end              INTEGER NOT NULL,
  is_quarter_end            INTEGER NOT NULL,
  is_year_end               INTEGER NOT NULL,
  iso_year                  INTEGER NOT NULL,
  iso_week                  INTEGER NOT NULL,
  is_week_start             INTEGER NOT NULL,
  is_week_end               INTEGER NOT NULL,
  is_business_day_generic   INTEGER NOT NULL,
  business_day_seq_generic  INTEGER NOT NULL
);
""")
conn.commit()

# Insert Data
rows = list(df[cols].itertuples(index=False, name=None))
cur.executemany(f"""
INSERT INTO dim_dates ({", ".join(cols)}) VALUES ({", ".join(["?"]*len(cols))});
""", rows)

# Create Index for Join Performance
cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_dim_dates_date_id ON dim_dates(date_id);")

conn.commit()
conn.close()

print("Script finished successfully.")
print(f"Database file is at: {sqlite_path.resolve()}")


Checking for and removing existing 'datadim_dates_v2.sqlite' table...
Table removed successfully.
Connecting to database and populating table...
Script finished successfully.
Database file is at: G:\My Drive\jpo\fin\data\fred\datadim_dates_v2.sqlite


In [4]:
print(df.head())  # Display the first few rows of the DataFrame for verification

         date   date_id  day_of_week day_name  is_weekend  year  quarter  \
0  1980-01-01  19800101            2      TUE           0  1980        1   
1  1980-01-02  19800102            3      WED           0  1980        1   
2  1980-01-03  19800103            4      THU           0  1980        1   
3  1980-01-04  19800104            5      FRI           0  1980        1   
4  1980-01-05  19800105            6      SAT           1  1980        1   

   month  day month_name  year_month  is_month_end  is_quarter_end  \
0      1    1    January      198001             0               0   
1      1    2    January      198001             0               0   
2      1    3    January      198001             0               0   
3      1    4    January      198001             0               0   
4      1    5    January      198001             0               0   

   is_year_end  iso_year  iso_week  is_week_start  is_week_end  \
0            0      1980         1              0       